In [1]:
import sys, os

# Enable importing pysparky
sys.path.append(os.pardir)

In [2]:
import pyspark
from pyspark.sql import SparkSession, DataFrame, Column
from pyspark.sql import functions as F, types as T

print(pyspark.__version__)

spark = SparkSession.builder.getOrCreate()

3.5.2


24/09/24 15:25:55 WARN Utils: Your hostname, codespaces-0aafae resolves to a loopback address: 127.0.0.1; using 10.0.0.163 instead (on interface eth0)
24/09/24 15:25:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 15:25:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

data_pdf = pd.read_csv(
    "https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv",
    names=["first_name", "last_name", "address", "region", "code", "postcode"],
)
data_sdf = spark.createDataFrame(data_pdf)

,first_name,last_name,address,region,code,postcode
0,John,Doe,120 jefferson st.,Riverside,NJ,8075
1,Jack,McGinnis,220 hobo Av.,Phila,PA,9119
2,"John ""Da Man""",Repici,120 Jefferson St.,Riverside,NJ,8075
3,Stephen,Tyler,"7452 Terrace ""At the Plaza"" road",SomeTown,SD,91234
4,NaN,Blankman,NaN,SomeTown,SD,298
5,"Joan ""the bone"", Anne",Jet,"9th, at Terrace plc",Desert City,CO,123


In [5]:
data_sdf.show()

+--------------------+---------+--------------------+-----------+----+--------+
|          first_name|last_name|             address|     region|code|postcode|
+--------------------+---------+--------------------+-----------+----+--------+
|                John|      Doe|   120 jefferson st.|  Riverside|  NJ|    8075|
|                Jack| McGinnis|        220 hobo Av.|      Phila|  PA|    9119|
|       John "Da Man"|   Repici|   120 Jefferson St.|  Riverside|  NJ|    8075|
|             Stephen|    Tyler|7452 Terrace "At ...|   SomeTown|  SD|   91234|
|                 NaN| Blankman|                 NaN|   SomeTown|  SD|     298|
|Joan "the bone", ...|      Jet| 9th, at Terrace plc|Desert City|  CO|     123|
+--------------------+---------+--------------------+-----------+----+--------+



In [22]:
import functools
from pysparky.transformation_ext import transforms, apply_cols
from pysparky.schema_ext import filter_columns_by_datatype

upper_cols_partial = functools.partial(apply_cols, col_func=F.upper)

In [23]:
pipeline = (
    (
        upper_cols_partial,
        {"cols": data_sdf.schema.filter_columns_by_datatype(T.StringType)},
    ),
    (lambda sdf: sdf.withColumn("postcode_plus_1", F.col("postcode") + 1), {}),
)

In [24]:
data_sdf.transforms(pipeline).show()

+--------------------+---------+--------------------+-----------+----+--------+---------------+
|          first_name|last_name|             address|     region|code|postcode|postcode_plus_1|
+--------------------+---------+--------------------+-----------+----+--------+---------------+
|                John|      Doe|   120 jefferson st.|  Riverside|  NJ|    8075|           8076|
|                Jack| McGinnis|        220 hobo Av.|      Phila|  PA|    9119|           9120|
|       John "Da Man"|   Repici|   120 Jefferson St.|  Riverside|  NJ|    8075|           8076|
|             Stephen|    Tyler|7452 Terrace "At ...|   SomeTown|  SD|   91234|          91235|
|                 NaN| Blankman|                 NaN|   SomeTown|  SD|     298|            299|
|Joan "the bone", ...|      Jet| 9th, at Terrace plc|Desert City|  CO|     123|            124|
+--------------------+---------+--------------------+-----------+----+--------+---------------+

